# Meta Classes
---
### Case 1
**Checking a function exist in a library**

In [16]:
# Library
class Base:
    def base_foo(self):
        return 'this is a sample function from base_foo()'

In [17]:
# My Code
class Derived(Base):
    def my_bar(self):
        return 'my_bar: ' + self.base_foo()

In [18]:
d = Derived()

In [19]:
d.my_bar()

'my_bar: this is a sample function from base_foo()'

**How do we ensure library Base class has `base_foo()` function**

In [20]:
# Library
class Base:
    def base_foox(self): # randomly changed the base_foo function name
        return 'this is a sample function from base_foo()'

**Checking using `assert` whether Base class has `base_foo` function or not**

In [21]:
assert hasattr(Base, 'base_foo') 

# My Code
class Derived(Base):
    def my_bar(self):
        return 'my_bar: ' + self.base_foo()

AssertionError: 

🔝 **throws `AssertionError` if `base_foo` not found in `Base` class and class `Deriverd` will never created**

---

### Case 2

**How do you ensure some `xyz` function is implemented in Derived class as your class is being base class**

In [25]:
# Your Code/Class
# library.py

class Base:
    def my_fun(self):
        return 'from Base to -> ' + self.most_critical_fun()

**To use `Base` class as base class and somebody is using `my_fun` in their code the `Derived` class code should have `most_critical_fun` definition**

In [29]:
class Deriverd(Base):
    def some_random_fun(self):
        return self.my_fun()

In [30]:
d = Deriverd()
d.some_random_func()

AttributeError: 'Deriverd' object has no attribute 'some_random_func'

> 🔝🔝 **throws an Error because `Base.my_fun` is expecting a definition of `most_critical_fun` in derived class**

---
### Solution 1
**The metaclass implementation**

In [65]:
class MetaBase(type):
    def __new__(cls, name, base_classes, body_dict):
        """
            cls: The MetaBase it self
            name: the name of the class where it triggered from
            base_classes: Tuple of base classes
                for BaseX: None
                for Derived: <class 'BaseX'>
                body_dict: {
                                '__module__': 'base_module',
                                '__qualname__': 'BaseX',
                                'base_bar': <function BaseX.base_bar at 0x10eb4a488>,
                                'base_foo': <function BaseX.base_foo at 0x10eb4a510>
                            }
        """
#        This will also work but hard coded name
#         if name != 'BaseX' and 'most_critical_fun' not in body:

        if base_classes and 'most_critical_fun' not in body_dict:
            raise TypeError('You have to implement most_critical_fun()')
        return super().__new__(cls, name, base_classes, body_dict)

In [71]:
class BaseX(metaclass=MetaBase):
    def base_bar(self):
        return 'base_bar from Base class'
    
    def base_foo(self):
        return 'this is Base.foo() and the critical function return value >> ' + self.most_critical_fun()

In [72]:
class DerivedClass1(BaseX):
    pass

TypeError: You have to implement most_critical_fun()

> 🔝🔝 **throws an `TypeError` because it checks the condition in `metaclass definition`**

In [73]:
class DerivedClass2(BaseX):
    def hey_random(self):
        return 'this is random'
    
    def most_critical_fun(self):
        return 'The most critical function....'

> 🔝🔝 **Doesn't throws an `TypeError` because we defined `most_critical_fun` in derived class `DerivedClass2`**

In [75]:
d2 = DerivedClass2()
d2.base_foo()

'this is Base.foo() and the critical function return value >> The most critical function....'